In [8]:
pip install datetime


   ---------------------------------------- 0.0/52.6 kB ? eta -:--:--
   ---------------------------------------- 52.6/52.6 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
from pyspark.sql import SparkSession
import os
import sys
from config import API_KEY
from datetime import datetime, timedelta
import requests


In [2]:
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"PATH: {os.environ.get('PATH')}")
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-1.8'
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")

JAVA_HOME: None
PATH: c:\Users\lucas\anaconda3;C:\Users\lucas\anaconda3;C:\Users\lucas\anaconda3\Library\mingw-w64\bin;C:\Users\lucas\anaconda3\Library\usr\bin;C:\Users\lucas\anaconda3\Library\bin;C:\Users\lucas\anaconda3\Scripts;C:\Users\lucas\anaconda3\bin;C:\Users\lucas\anaconda3\condabin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Rockwell Software\RSCommon;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Program Files\Git\cmd;C:\Program Files\MongoDB\Se

In [7]:
spark = SparkSession.builder \
    .appName("weatherapp") \
    .getOrCreate()

In [12]:
url = "http://api.weatherapi.com/v1/history.json"
api_key = API_KEY
city = 'Philadelphia'
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)


params = {
    'key': api_key,
    'q': city,
    'dt': '2024-10-22' 
}

response = requests.get(url,params=params)
data = response.json()



In [13]:

data

{'location': {'name': 'Philadelphia',
  'region': 'Pennsylvania',
  'country': 'United States of America',
  'lat': 39.9522,
  'lon': -75.1642,
  'tz_id': 'America/New_York',
  'localtime_epoch': 1732334937,
  'localtime': '2024-11-22 23:08'},
 'forecast': {'forecastday': [{'date': '2024-10-22',
    'date_epoch': 1729555200,
    'day': {'maxtemp_c': 29.1,
     'maxtemp_f': 84.4,
     'mintemp_c': 13.5,
     'mintemp_f': 56.3,
     'avgtemp_c': 21.1,
     'avgtemp_f': 70.0,
     'maxwind_mph': 6.3,
     'maxwind_kph': 10.1,
     'totalprecip_mm': 0.0,
     'totalprecip_in': 0.0,
     'totalsnow_cm': 0.0,
     'avgvis_km': 10.0,
     'avgvis_miles': 6.0,
     'avghumidity': 54,
     'daily_will_it_rain': 0,
     'daily_chance_of_rain': 0,
     'daily_will_it_snow': 0,
     'daily_chance_of_snow': 0,
     'condition': {'text': 'Sunny',
      'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
      'code': 1000},
     'uv': 7.0},
    'astro': {'sunrise': '07:19 AM',
     'sunset': '

In [16]:
data['location']['name']

'Philadelphia'

In [18]:
data['forecast']

{'forecastday': [{'date': '2024-10-22',
   'date_epoch': 1729555200,
   'day': {'maxtemp_c': 29.1,
    'maxtemp_f': 84.4,
    'mintemp_c': 13.5,
    'mintemp_f': 56.3,
    'avgtemp_c': 21.1,
    'avgtemp_f': 70.0,
    'maxwind_mph': 6.3,
    'maxwind_kph': 10.1,
    'totalprecip_mm': 0.0,
    'totalprecip_in': 0.0,
    'totalsnow_cm': 0.0,
    'avgvis_km': 10.0,
    'avgvis_miles': 6.0,
    'avghumidity': 54,
    'daily_will_it_rain': 0,
    'daily_chance_of_rain': 0,
    'daily_will_it_snow': 0,
    'daily_chance_of_snow': 0,
    'condition': {'text': 'Sunny',
     'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
     'code': 1000},
    'uv': 7.0},
   'astro': {'sunrise': '07:19 AM',
    'sunset': '06:10 PM',
    'moonrise': '10:18 PM',
    'moonset': '01:28 PM',
    'moon_phase': 'Waning Gibbous',
    'moon_illumination': 72},
   'hour': [{'time_epoch': 1729569600,
     'time': '2024-10-22 00:00',
     'temp_c': 19.3,
     'temp_f': 66.8,
     'is_day': 0,
     'condition': 